In [31]:
import simpy
import random
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import math

%matplotlib inline

In [6]:
CARRILES_NORMALESS = 2
CARRILES_PESADOS = 1
TIEMPO_LLEGADA = 3

TIEMPO_EJECUCION = 30 * 24 * 60 # DIAS * HORAS * MINUTOS



In [41]:
def tiempo_revision():
    return int(np.random.normal(20,10,1)[0])

def seleccion_vehiculo():
    return np.random.choice([1,2], p=[0.15,0.85])#1=pesado   y 2=normal

def pasan_revision():
    return np.random.choice([1,2], p=[0.6,0.4])#1=si   y 2=no


In [42]:
vehiculos_rechasados = {} # hora que regresa , tipo de vehiculo


cantidad = {}


vehiculos_dia = {}


class negocio(object):
    def __init__(self, env, num_carriles):
        self.env = env
        self.carriles = simpy.Resource(env, num_carriles)

    def revision(self, tiempo_revision):
        yield self.env.timeout(tiempo_revision)

        


def llegada_vehiculo(env,negocio,tipo):
    with negocio.carriles.request() as carril:
        yield carril

        #tiempo de llegada
        yield env.timeout(random.randint(3,5))
        #tiempo cuando saca el turno
        yield env.timeout(random.randint(1,3))
        #tiempo cuando paga la matricula
        yield env.timeout(random.randint(5,10))

        #hacer un if para ver si consiguio o no terminar el proceso para darles sello
        t = tiempo_revision()
        while t < 0:
            t = tiempo_revision()
        yield env.process(negocio.revision(t))

        #para saber si pasa o no la revision
        res = pasan_revision()
        if(res==1):# si paso la revision
            yield env.timeout(random.randint(1,2)) # sello
            if(tipo in cantidad):
                cantidad[tipo] += 1
            else:
                cantidad[tipo] = 1
            
            #cantidad de revisiones por dia
            tiempo_dia = env.now
            dia = math.trunc(tiempo_dia/1440)
            if(dia in vehiculos_dia):
                vehiculos_dia[dia] += 1
            else:
                vehiculos_dia[dia] = 1

        else:
            tiempo_volver = env.now + 2880
            vehiculos_rechasados[tiempo_volver] = tipo
        #si cuando un vehiculo es rechazado y no cuenta el tiempo de salida, meter esto dentro de un if
        #tiempo de salida
        yield env.timeout(random.randint(5,10))

def ejecutar_simulacion(env, num_carriles_normal, num_carriles_pesados):
    carriles_normales = negocio(env, num_carriles_normal)
    carril_pesado = negocio(env, num_carriles_pesados)

    #generacion de llegada de carros
    tie_llegada = env.now
    while True:
        k = env.now
        if k in vehiculos_rechasados:
            if vehiculos_rechasados[k]==1:
                env.process(llegada_vehiculo(env,carril_pesado,vehiculos_rechasados[k]))
            else:
                env.process(llegada_vehiculo(env,carriles_normales,vehiculos_rechasados[k]))


        if(tie_llegada==k):
            tipo = seleccion_vehiculo()
            if(tipo==1):
                env.process(llegada_vehiculo(env,carril_pesado,tipo))
            else:
                env.process(llegada_vehiculo(env,carriles_normales,tipo))
            tie_llegada=k+3
        
        yield env.timeout(1)


random.seed(77)

env = simpy.Environment()
env.process(ejecutar_simulacion(env,CARRILES_NORMALESS,CARRILES_PESADOS))

env.run(until = TIEMPO_EJECUCION)

print('Total pesados revisados: %s' % cantidad[1])
print('Total normales revisados: %s' % cantidad[2])
print('Total revisados: %s' % (cantidad[1]+cantidad[2]))
print('Total rechasados: %s' % len(vehiculos_rechasados))


for x in vehiculos_dia:
    print('Total revisados el dia %s: %s' % (x,vehiculos_dia[x]))

print('Tomando en cuenta que los vehiculos rechasados luego fueron revisados nuevamente y terminaron este proceos, se obtuvieron los siguientes resultados: ')

porcentaje = (len(vehiculos_rechasados)*100)/(cantidad[1]+cantidad[2])
print('Porcentaaje de vehiculos rechazados: ' + str(porcentaje))
print('Porcentaje de vehiculos revisados de manera directa: ' + str(100-porcentaje))


Total pesados revisados: 647
Total normales revisados: 1224
Total revisados: 1871
Total rechasados: 1217
Total revisados el dia 0: 65
Total revisados el dia 1: 55
Total revisados el dia 2: 59
Total revisados el dia 3: 68
Total revisados el dia 4: 69
Total revisados el dia 5: 55
Total revisados el dia 6: 64
Total revisados el dia 7: 64
Total revisados el dia 8: 57
Total revisados el dia 9: 67
Total revisados el dia 10: 60
Total revisados el dia 11: 59
Total revisados el dia 12: 59
Total revisados el dia 13: 58
Total revisados el dia 14: 72
Total revisados el dia 15: 56
Total revisados el dia 16: 60
Total revisados el dia 17: 62
Total revisados el dia 18: 63
Total revisados el dia 19: 61
Total revisados el dia 20: 64
Total revisados el dia 21: 65
Total revisados el dia 22: 63
Total revisados el dia 23: 67
Total revisados el dia 24: 65
Total revisados el dia 25: 61
Total revisados el dia 26: 62
Total revisados el dia 27: 66
Total revisados el dia 28: 65
Total revisados el dia 29: 60
Toman

In [28]:
2880%1440

0

In [30]:
2880/1440

2.0

In [32]:
math.trunc(2779/1440)

1